<a href="https://colab.research.google.com/github/michelleli246/ConnectionChecker/blob/master/capstone_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# check the current python version
import sys
print(sys.version)
!pip install folium
!pip install tqdm
!pip install numpy
!pip install pandas
!pip install nltk
!pip install scikit-learn
!pip install scipy
!pip install geopy
!pip install tabulate
!pip install textblob
!pip install spacy
!python -m spacy download en_core_web_sm

#Counter
from collections import Counter

#Operation
import operator

#Natural Language Processing Packages
import re
import nltk

## Download Resources
nltk.download("vader_lexicon")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tag import PerceptronTagger
from nltk.data import find

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
     |████████████████████████████████| 12.0 MB 4.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
import pandas as pd


In [3]:
test_indicators = pd.read_csv('testindicators.csv')

In [ ]:
#data= pd.read_csv('IRIS.csv')

In [ ]:
#data.head()

In [4]:
def get_stop_words():
    stop = set(stopwords.words('english'))
    stop.remove('during') # include during for "during the reporting period"
    return stop

In [83]:
sentence_re = r'''(?x)          # set flag to allow verbose regexps
        (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
      | \w+(?:-\w+)*        # words with optional internal hyphens
      | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
      | \.\.\.              # ellipsis
      | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
    '''

# Part of Speech Tagging
# Google: https://en.wikipedia.org/wiki/Part-of-speech_tagging
tagger = PerceptronTagger()
pos_tag = tagger.tag

noun_grammar = r"""
    NBAR:
        {<NN.*>+}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
        {<NN>+}

    VP:
        {<VBG>}
    
    IN:
        {<IN>}
"""

noun_grammar = r"""
    NBAR:
        {<NN.*>+}
    
    NP:
        {<IN><VBG><NN>}
        {<NBAR>}

    VP:
        {<VB|VBG|VBD|VBN>}
    
"""

chunker = nltk.RegexpParser(noun_grammar)
lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()
stopword_list = get_stop_words()
  
def normalise(word,lemmatizer=lemmatizer, stemmer=stemmer):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    #word = stemmer.stem(word)
    word = lemmatizer.lemmatize(word)
    return word

def acceptable_word(word, stopword_list=stopword_list):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(1 < len(word) <= 40 and word.lower() not in stopword_list)
    return accepted

def flatten_phrase_lists(npTokenList):
    finalList =[]
    for phrase in npTokenList:
        token = ''
        for word in phrase:
            token += word + ' '
        finalList.append(token.rstrip())
    return finalList

def getPhrases(text):

    counter = Counter()

    #test = flatten_phrase_lists(chunker.parse(pos_tag(re.findall(r'\w+', text))))
    test = chunker.parse(pos_tag(re.findall(r'\w+', text)))

    return test

In [27]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
'''
order of operations:
1. tokenize( remove punctuation) pos tag
2. chunk noun-noun phrases or adjective-noun 
3. remove stop words?
4. create directed graph - edges are the verbs
'''

'\norder of operations:\n1. tokenize( remove punctuation) pos tag\n2. chunk noun-noun phrases or adjective-noun \n3. remove stop words?\n4. create directed graph - edges are the verbs\n'

## POS tagging w positions

In [80]:
def POS_tagger(subtrees):
  terms = {}
  lvl1 = 0
  for test in subtrees:
    phrase = test.label()
    if phrase == 'NP':
      p = {}
      for w in range(len(test.leaves())):
        p[w] = test.leaves()[w][0]
      terms[lvl1] = {'pos':'N', 'phrase':p}
      lvl1+=1
    elif phrase == 'VP':
      p = {}
      for w in range(len(test.leaves())):
        p[w] = test.leaves()[w][0]
      terms[lvl1] = {'pos':'V', 'phrase':p}

      lvl1+=1
    elif phrase == 'IN':
      p = {}
      for w in range(len(test.leaves())):
        p[w] = test.leaves()[w][0]
      terms[lvl1] = {'pos':'IN', 'phrase':p}
      lvl1+=1

  return terms

In [81]:
def get_key_pos_tag(phrase):
  phrases = [normalise(i) for i in word_tokenize(phrase) if acceptable_word(i)]
  print('p',phrases)
  n_phrases = getPhrases(' '.join(phrases))
  print('nphrase', n_phrases)
  terms = POS_tagger(n_phrases.subtrees())
  print('terms', terms)

  noun_phrases = []
  verb_phrases = []
  for k,v in terms.items():
    if v['pos'] == 'N':
      noun_phrases.append(v)
    if v['pos'] == 'V':
      verb_phrases.append(v)

  return noun_phrases, verb_phrases

In [84]:
get_key_pos_tag('Value of payments made to smallholder farmers who sold to the organization during the reporting period.')

p ['value', 'payment', 'made', 'smallholder', 'farmer', 'sold', 'organization', 'during', 'reporting', 'period']
nphrase (S
  (NP (NBAR value/NN payment/NN))
  (VP made/VBD)
  smallholder/JJR
  (NP (NBAR farmer/NN))
  (VP sold/VBN)
  (NP (NBAR organization/NN))
  during/IN
  (VP reporting/VBG)
  (NP (NBAR period/NN)))
terms {0: {'pos': 'N', 'phrase': {0: 'value', 1: 'payment'}}, 1: {'pos': 'V', 'phrase': {0: 'made'}}, 2: {'pos': 'N', 'phrase': {0: 'farmer'}}, 3: {'pos': 'V', 'phrase': {0: 'sold'}}, 4: {'pos': 'N', 'phrase': {0: 'organization'}}, 5: {'pos': 'V', 'phrase': {0: 'reporting'}}, 6: {'pos': 'N', 'phrase': {0: 'period'}}}


([{'phrase': {0: 'value', 1: 'payment'}, 'pos': 'N'},
  {'phrase': {0: 'farmer'}, 'pos': 'N'},
  {'phrase': {0: 'organization'}, 'pos': 'N'},
  {'phrase': {0: 'period'}, 'pos': 'N'}],
 [{'phrase': {0: 'made'}, 'pos': 'V'},
  {'phrase': {0: 'sold'}, 'pos': 'V'},
  {'phrase': {0: 'reporting'}, 'pos': 'V'}])

In [32]:
test_indicators['parsed_pos'] =  test_indicators['IndicatorText'].apply(get_key_pos_tag)

In [19]:
pd.set_option('display.max_colwidth', None)

In [61]:
test_indicators[['IndicatorText','parsed_pos']].head(20)

,IndicatorText,parsed_pos
0,Number of unique smallholder farmer individuals who were clients during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'number'}}, {'pos': 'N', 'phrase': {0: 'smallholder', 1: 'farmer'}}, {'pos': 'N', 'phrase': {0: 'client'}}, {'pos': 'N', 'phrase': {0: 'during', 1: 'reporting', 2: 'period'}}], [])"
1,Number of smallholder farmers who sold to the organization during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'number', 1: 'smallholder', 2: 'farmer'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'during', 1: 'reporting', 2: 'period'}}], [])"
2,Value of payments made to smallholder farmers who sold to the organization during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'value', 1: 'payment'}}, {'pos': 'N', 'phrase': {0: 'farmer'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'during', 1: 'reporting', 2: 'period'}}], [])"
3,Percentage of payments made to smallholder farmer suppliers compared to total payments made to all suppliers of the organization during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'percentage', 1: 'payment'}}, {'pos': 'N', 'phrase': {0: 'farmer', 1: 'supplier'}}, {'pos': 'N', 'phrase': {0: 'payment'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'during', 1: 'reporting', 2: 'period'}}], [])"
4,Type of crop(s) produced by the organization during the reporting period. Select from the options in the Reference List tab.,"([{'pos': 'N', 'phrase': {0: 'type', 1: 'crop'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'during', 1: 'reporting', 2: 'period', 3: 'select', 4: 'option', 5: 'reference', 6: 'list', 7: 'tab'}}], [])"
5,Type of livestock product(s) produced by the organization during the reporting period. Select from the options in the Reference List tab.,"([{'pos': 'N', 'phrase': {0: 'type', 1: 'livestock', 2: 'product'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'during', 1: 'reporting', 2: 'period', 3: 'select', 4: 'option', 5: 'reference', 6: 'list', 7: 'tab'}}], [])"
6,Area of land directly controlled by the organization during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'area', 1: 'land'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'during', 1: 'reporting', 2: 'period'}}], [])"
7,Area of land directly controlled by the organization and under cultivation. Report directly controlled land area cultivated during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'area', 1: 'land'}}, {'pos': 'N', 'phrase': {0: 'organization', 1: 'cultivation', 2: 'report'}}, {'pos': 'N', 'phrase': {0: 'land', 1: 'area'}}, {'pos': 'N', 'phrase': {0: 'reporting', 1: 'period'}}], [])"
8,Area of land directly controlled by the organization and under sustainable cultivation or sustainable stewardship. Report directly controlled land area sustainably managed during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'area', 1: 'land'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'cultivation'}}, {'pos': 'N', 'phrase': {0: 'report'}}, {'pos': 'N', 'phrase': {0: 'land', 1: 'area'}}, {'pos': 'N', 'phrase': {0: 'reporting', 1: 'period'}}], [])"
9,Area of land directly controlled by the organization and treated with pesticides. Report directly controlled land area treated during the reporting period.,"([{'pos': 'N', 'phrase': {0: 'area', 1: 'land'}}, {'pos': 'N', 'phrase': {0: 'organization'}}, {'pos': 'N', 'phrase': {0: 'report'}}, {'pos': 'N', 'phrase': {0: 'land', 1: 'area'}}, {'pos': 'N', 'phrase': {0: 'reporting', 1: 'period'}}], [])"


## Positional Rule Based System

In [ ]:
#measurement is the first noun
#phenomenon is the second noun
measurement = nouns[0]
phenomenon = nouns[1]

print(measurement)
print(phenomenon)

{'pos': 'N', 'phrase': {0: 'number'}}
{'pos': 'N', 'phrase': {0: 'smallholder', 1: 'farmer'}}


In [ ]:
# reporting period and it's associated IN is the reporting period
rep_period_detail = other[-1]
rep_period = nouns[-1]

In [ ]:
print(rep_period_detail)
print(rep_period)

{'pos': 'IN', 'phrase': {0: 'during'}}
{'pos': 'N', 'phrase': {0: 'period'}}


## Spacy Named Entity Recog

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Number of unique smallholder farmer individuals who were clients during the reporting period.")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

## Gensim Word Vectors

In [ ]:
import gensim.downloader as api

# download the pretrained embeddings
pre_ft_vectors = api.load("glove-wiki-gigaword-100")
#pre_ft_vector2 = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
# indicator keyword vectors
indicator_vectors = {}
for n in nouns:
  p = n['phrase'].values()
  for w in p:
    v = pre_ft_vectors.get_vector(w)
    indicator_vectors[w] = v


In [ ]:
# cids class vectors
cids_classes = ['indicator','organization','person','measurement','stakeholder']

cids_vectors = {}

for w in cids_classes:
  v = pre_ft_vectors.get_vector(w)
  cids_vectors[w] = v

In [ ]:
# cosine distance
from scipy.spatial import distance

In [ ]:
i = indicator_vectors['client']

cos_distances = []
for c in cids_classes:
  cos_distances.append(distance.cosine(cids_vectors[c],i))

cos_distances

[0.8884387165307999,
 0.6253991723060608,
 0.42667192220687866,
 0.7757274955511093,
 0.887111522257328]